## Setup

In [1]:
import os
import csv
from tqdm.notebook import tqdm
import numpy as np
import math
import torch
from IPython import display
import subprocess
import VARIABLES
import importlib
importlib.reload(VARIABLES)
from VARIABLES import *


In [2]:
!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.1.47  Python-3.12.3 torch-2.2.2+cu121 CUDA:0 (NVIDIA GeForce RTX 3050, 8192MiB)
Setup complete  (32 CPUs, 31.8 GB RAM, 602.6/930.7 GB disk)


In [3]:
!pip install supervision

from IPython import display
display.clear_output()

import supervision as sv
print("supervision.__version__:", sv.__version__)

supervision.__version__: 0.19.0


In [10]:
from ultralytics import YOLO

model_byke = YOLO("yolov8x.pt")
model_byke.fuse()

model_scooter = YOLO("runs/detect/train16/weights/best.pt")
model_scooter.fuse()

models = {
    "escooter": (model_scooter, [1]),
    "bicycle": (model_byke, [1])
    }

# dict maping class_id to class_name

# class_ids of interest - car, motorcycle, bus and truck

# model_byke.model.names


YOLOv8x summary (fused): 268 layers, 68200608 parameters, 0 gradients, 257.8 GFLOPs
Model summary (fused): 268 layers, 68125494 parameters, 0 gradients, 257.4 GFLOPs


## Utils

In [11]:
import VARIABLES
import importlib
importlib.reload(VARIABLES)
from VARIABLES import *

import utils
from utils import *
importlib.reload(utils)
from utils import *

videos_folder = "probando4"
videos_folder_path = os.path.join("full_recordings", videos_folder)


results_path = os.path.join("processing_results", f"{getDateTimeShort()} {videos_folder}")
counts_path = os.path.join(results_path, "counts")
videos_output_path = os.path.join(results_path, "videos_output")


os.makedirs(results_path, exist_ok=True)
os.makedirs(counts_path, exist_ok=True)
os.makedirs(videos_output_path, exist_ok=True)

max_counters = {
  class_name: LineZoneMaxCounterHelper(class_id[0], class_name) 
  for class_name, (_, class_id) in models.items()
}


vih = VideoInfoHandler(models)

for file_name in tqdm(os.listdir(videos_folder_path), desc=" Videos remaining", position=0):
  file_path = os.path.join(videos_folder_path, file_name)
  vih.re_init(file_path)
  process_video(
    source_path = file_path,
    target_path = os.path.join(videos_output_path, file_name),
    models = models,
    callback = callback,
    vih = vih
  )
  
  for max_c in max_counters.values():
    max_c.update_counting(vih.line_zones, file_name)


save_results(max_counters, counts_path)


 Videos remaining:   0%|          | 0/1 [00:00<?, ?it/s]

 Video processing:   0%|          | 0/922 [00:00<?, ?it/s]

In [ ]:
vih.line_zones[4].class_out_count

defaultdict(int, {('bicycle', 1): 28, ('escooter', 1): 0})